<a href="https://colab.research.google.com/github/Deepak250104/Twitter_Sentiment_Analysis/blob/main/Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Getting all the dependenscies**

Uploadin the kaggle.json file

In [1]:
!pip install kaggle

In [2]:
#configuring the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600  ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


Importing Twitter Sentiment dataset from Kaggle

In [3]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 99% 80.0M/80.9M [00:04<00:00, 24.1MB/s]
100% 80.9M/80.9M [00:04<00:00, 18.4MB/s]


Extracting the compressed dataset

In [4]:
from zipfile import ZipFile
data = '/content/sentiment140.zip'

with ZipFile(data, 'r') as zip:
  zip.extractall()
  print("Files sucessfully extracted.")

Files sucessfully extracted.


Importing all the necessary modules

In [5]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

**Stopwords** are common words (e.g., "the", "is", "in", "and") that do not add much meaning to a text but appear frequently.

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# **Data** **Processing**

In [7]:
# Naming the columns
column_names = ['target', 'id', 'date', 'flag', 'user', 'text']

# Loading the data from csv file to pandas dataframe
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names= column_names, encoding='ISO-8859-1')

In [8]:
# Number of rows and columns
twitter_data.shape

(1600000, 6)

In [9]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


Finding missing values

In [10]:
twitter_data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


Checking the distribution of target column

In [11]:
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


Converting the target "4" to "1"

In [12]:
twitter_data.replace({'target':{4:1}}, inplace = True)

In [13]:
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


Here,
**"0"** means **negative** tweet &
**"1"** means **positive:** tweet.

**Stemming**

Stemming is the process of reducing a word to its root form by removing suffixes. It helps normalize words in NLP by treating different forms of a word as the same.

In [14]:
stem = PorterStemmer()

In [15]:
def stemming(tweet):
  stemmed_tweet = re.sub('[^a-zA-Z]', ' ', tweet) # Removing everything that is not letters
  stemmed_tweet = stemmed_tweet.lower().split()
  stemmed_content = [stem.stem(word) for word in stemmed_tweet if not word in stopwords.words('english')]
  stemmed_tweet = ' '.join(stemmed_content)

  return stemmed_tweet

Creating a new column in the dataframe called "stemmed_content"

In [16]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [17]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [18]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [19]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


Seperating the data and the label.

In [20]:
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [21]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [22]:
print(Y)

[0 0 0 ... 1 1 1]


Splitting the data into training data and test data.

In [23]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [24]:
print(X_train)

['bad feel' 'love weekend' 'littlew bit far day trip fun get muddi today'
 ... 'twitter follow reform famou peopl'
 'aria cut hand open morn say like mommi anymor'
 'charlii sweet im excit aww miss u guy today']


In [25]:
print(X_test)

['brodiejay oh im go wow mona vale real place afteral know suck mvill slow train pffft'
 'babi grow' 'paint black roll stone best' ...
 'belladonna miss good music' 'reec r know imma get'
 'ooooooohnoooooo forgot charg ipod work one sad littl bar left']


Converting textual data into numerical data.

In [26]:
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [27]:
print(X_train)

  (0, 31233)	0.7519171067329923
  (0, 129956)	0.6592576617698773
  (1, 241525)	0.5942540519072339
  (1, 437910)	0.8042773910733999
  (2, 237072)	0.6367700241935805
  (2, 42838)	0.2294043092907454
  (2, 128288)	0.2817039475788181
  (2, 93639)	0.16612263205577085
  (2, 416219)	0.2970321685021027
  (2, 140234)	0.2237237868613854
  (2, 145727)	0.16444046079173227
  (2, 278606)	0.48531397707654644
  (2, 411086)	0.18464287904169108
  (3, 201928)	0.524207488360723
  (3, 308910)	0.41292329558269175
  (3, 179097)	0.4788299006944713
  (3, 445747)	0.2188627839232472
  (3, 8560)	0.37668420201866026
  (3, 286683)	0.3682809811482607
  (4, 457632)	0.7604081439946883
  (4, 302738)	0.35966531233864635
  (4, 315347)	0.3291134109458034
  (4, 415818)	0.27525076285252903
  (4, 227660)	0.3291530007938817
  (5, 93639)	0.181548748438646
  :	:
  (1279996, 132233)	0.2452899533029938
  (1279996, 452298)	0.569688609447451
  (1279996, 34420)	0.41424149821985934
  (1279997, 135226)	0.3062578356465206
  (1279997, 42

In [28]:
print(X_train)

  (0, 31233)	0.7519171067329923
  (0, 129956)	0.6592576617698773
  (1, 241525)	0.5942540519072339
  (1, 437910)	0.8042773910733999
  (2, 237072)	0.6367700241935805
  (2, 42838)	0.2294043092907454
  (2, 128288)	0.2817039475788181
  (2, 93639)	0.16612263205577085
  (2, 416219)	0.2970321685021027
  (2, 140234)	0.2237237868613854
  (2, 145727)	0.16444046079173227
  (2, 278606)	0.48531397707654644
  (2, 411086)	0.18464287904169108
  (3, 201928)	0.524207488360723
  (3, 308910)	0.41292329558269175
  (3, 179097)	0.4788299006944713
  (3, 445747)	0.2188627839232472
  (3, 8560)	0.37668420201866026
  (3, 286683)	0.3682809811482607
  (4, 457632)	0.7604081439946883
  (4, 302738)	0.35966531233864635
  (4, 315347)	0.3291134109458034
  (4, 415818)	0.27525076285252903
  (4, 227660)	0.3291530007938817
  (5, 93639)	0.181548748438646
  :	:
  (1279996, 132233)	0.2452899533029938
  (1279996, 452298)	0.569688609447451
  (1279996, 34420)	0.41424149821985934
  (1279997, 135226)	0.3062578356465206
  (1279997, 42

# Model Training

Logistic Regression model

In [29]:
model = LogisticRegression(max_iter=1000)

In [30]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

# Model Evaluation

Accuracy score on the training data

In [31]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print("Accuracy score on the training data: ", training_data_accuracy)

Accuracy score on the training data:  0.8023890625


In [32]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print("Accuracy score on the test data: ", test_data_accuracy)

Accuracy score on the test data:  0.77656875


Model Accuracy = 77.6%

# Saving and reusing the trained model

In [33]:
import pickle

Saving the model

In [34]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

Using the saved model

In [35]:
#loading the model
loaded_model = pickle.load(open('/content/trained_model.sav', 'rb'))

In [36]:
X_new = X_test[1234]
print(Y_test[1234])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print("Negative Tweet")
else:
  print("Positive Tweet")

1
[1]
Positive Tweet


In [37]:
X_new = X_test[565]
print(Y_test[565])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print("Negative Tweet")
else:
  print("Positive Tweet")

0
[0]
Negative Tweet
